# Filtro de Kalman

Funções de cálculo das distâncias medidas pelos sensores.

$dF (x, y, \theta) = \begin{matrix} \frac{|Sy - Yp|}{|\cos(\theta)|}, & 0 \le \theta \lt \frac{\pi}{2}  \\ 
          \frac{|Sx - Xp|}{|\sin(\theta)|}, & \frac{\pi}{2} \le \theta \lt \pi \\
          \frac{Yp}{|\cos(\theta)|}, & \pi \le \theta \lt \frac{3\pi}{2}  \\ 
          \frac{Xp}{|\sin(\theta)|}, & \frac{3\pi}{2} \le \theta \lt 2\pi
           \end{matrix} $

$dR (x, y, \theta) = \begin{matrix} \frac{|Sx - Xp|}{|\cos(\theta)|}, & 0 \le \theta \lt \frac{\pi}{2}  \\ 
          \frac{Yp}{|\sin(\theta)|}, & \frac{\pi}{2} \le \theta \lt \pi \\
          \frac{Xp}{|\cos(\theta)|}, & \pi \le \theta \lt \frac{3\pi}{2}  \\ 
          \frac{|Sy - Yp|}{|\sin(\theta)|}, & \frac{3\pi}{2} \le \theta \lt 2\pi
           \end{matrix} $

$dL (x, y, \theta) = \begin{matrix} \frac{Xp}{|\cos(\theta)|}, & 0 \le \theta \lt \frac{\pi}{2}  \\ 
          \frac{|Sy - Yp|}{|\sin(\theta)|}, & \frac{\pi}{2} \le \theta \lt \pi \\
          \frac{Sx - Xp}{|\cos(\theta)|}, & \pi \le \theta \lt \frac{3\pi}{2}  \\ 
          \frac{Yp}{|\sin(\theta)|}, & \frac{3\pi}{2} \le \theta \lt 2\pi
           \end{matrix} $

In [241]:
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [242]:
def determinarDF(x, y, theta, Sx, Sy):
    dF = 0
    if theta >= 0 and theta < np.pi / 2:
        dF = (Sy - y) / np.abs(np.cos(theta)) 
    elif theta >= np.pi / 2 and theta < np.pi:
        dF = (Sx - x) / np.abs(np.sin(theta))
    elif theta >= np.pi and theta < 3*np.pi/2:
        dF = y / np.abs(np.cos(theta)) 
    elif theta >= 3 * np.pi / 2 and theta < 2 * np.pi:
        dF = x / np.abs(np.sin(theta))        
    return dF

def determinarDR(x, y, theta, Sx, Sy):
    dR = 0
    if theta >= 0 and theta < np.pi / 2:
        dR = (Sx - x) / np.abs(np.cos(theta)) 
    elif theta >= np.pi / 2 and theta < np.pi:
        dR = (Sx - x) / np.abs(np.sin(theta))
    elif theta >= np.pi and theta < 3*np.pi/2:
        dR = x / np.abs(np.cos(theta)) 
    elif theta >= 3 * np.pi / 2 and theta < 2 * np.pi:
        dR = (Sy - y) / np.abs(np.sin(theta))        
    return dR

def determinarDL(x, y, theta, Sx, Sy):
    dL = 0
    if theta >= 0 and theta < np.pi / 2:
        dL = x / np.abs(np.cos(theta)) 
    elif theta >= np.pi / 2 and theta < np.pi:
        dL = (Sy - y) / np.abs(np.sin(theta))
    elif theta >= np.pi and theta < 3*np.pi/2:
        dL = (Sx - x) / np.abs(np.cos(theta)) 
    elif theta >= 3 * np.pi / 2 and theta < 2 * np.pi:
        dL = y / np.abs(np.sin(theta))        
    return dL

    
def determinarMatrizH(x, y, theta, Sx, Sy):
    if theta >= 0 and theta < np.pi / 2:
        H = np.array([[0, -1/np.abs(np.cos(theta)), 0, 0, (((Sy - y)*np.tan(theta)) / np.abs(np.cos(theta)))],
                       [-1/np.abs(np.cos(theta)), 0, 0, 0,(((Sx - x)*np.tan(theta)) / np.abs(np.cos(theta)))],
                       [-1/np.abs(np.cos(theta)), 0, 0, 0, (((x)*np.tan(theta)) / np.abs(np.cos(theta)))],
                       [0, 0, 0, 0, 0],
                       [0, 0, 0, 0, 0]])
    elif theta >= np.pi / 2 and theta < np.pi:
        H = np.array([[-1/np.abs(np.sin(theta)), 0, 0, 0, ((-(Sx - x)*np.cos(theta)) / (np.sin(theta) * np.abs(np.sin(theta))))],
                       [0, 1/np.abs(np.sin(theta)),  0, 0, ((-(y)*np.cos(theta)) / (np.sin(theta) * np.abs(np.sin(theta))))],
                       [0, -1/np.abs(np.sin(theta)), 0, 0, ((-(Sy - y)*np.cos(theta)) / (np.sin(theta) * np.abs(np.sin(theta))))],
                       [0, 0, 0, 0, 0],
                       [0, 0, 0, 0, 0]])
    elif theta >= np.pi and theta < 3*np.pi/2:
        H = np.array([[0, 1/np.abs(np.cos(theta)), 0, 0, (((y)*np.tan(theta)) / np.abs(np.cos(theta)))],
                       [1/np.abs(np.cos(theta)), 0, 0, 0, (((x)*np.tan(theta)) / np.abs(np.cos(theta)))],
                       [-1/np.abs(np.cos(theta)), 0, 0, 0, (((Sx - x)*np.tan(theta)) / np.abs(np.cos(theta)))],
                       [0, 0, 0, 0, 0],
                       [0, 0, 0, 0, 0]])
    elif theta >= 3 * np.pi / 2 and theta < 2 * np.pi:
        H = np.array([[1/np.abs(np.sin(theta)), 0,   0, 0, ((-(x)*np.cos(theta)) / (np.sin(theta) * np.abs(np.sin(theta))))],
                       [0, -1/np.abs(np.sin(theta)),  0, 0, ((-(Sy - y)*np.cos(theta)) / (np.sin(theta) * np.abs(np.sin(theta))))],
                       [0, -1/np.abs(np.sin(theta)), 0, 0, ((-(y)*np.cos(theta)) / (np.sin(theta) * np.abs(np.sin(theta))))],
                       [0, 0, 0, 0, 0],
                       [0, 0, 0, 0, 0]])
    return H

In [243]:
def calculateInverseSk(Sk):
    numeros = []
    matrixSk = []
    for i in range(0, 5):
        for j in range(0, 5):
            aux = Sk[i, j][0, 0]
            numeros.append(aux)
        matrixSk.append(numeros.copy())
        numeros.clear()
    novoSk = np.array(matrixSk)
    inversoSk = np.linalg.inv(novoSk)
    return inversoSk
    

In [244]:
def filtroDeKalman(posicao_anterior, deltaT, u_k, P, distancias_lidas):
    posicao_anterior = (np.array([posicao_anterior])).transpose()
    x_kmenos1, y_kmenos1, vx_kmenos1, vy_kmenos1, theta_kmenos1 = posicao_anterior[0], posicao_anterior[1], posicao_anterior[2], posicao_anterior[3], posicao_anterior[4] 
    aceleracao_medida, velocidade_angular = u_k[0], u_k[1]
    alpha = theta_kmenos1 + (deltaT*velocidade_angular) 
    deltaX = (1/2 * (deltaT ** 2) * aceleracao_medida  + vx_kmenos1*deltaT) * np.cos(alpha)
    deltaY = (1/2 * (deltaT ** 2) * aceleracao_medida + vy_kmenos1*deltaT) * np.sin(alpha)
    deltaVx = aceleracao_medida * deltaT * np.cos(alpha)
    deltaVy = aceleracao_medida * deltaT * np.sin(alpha)
    deltaTheta = deltaT * velocidade_angular
    posicao_predita = posicao_anterior + np.array([[deltaX],
                                                    [deltaY],
                                                    [deltaVx],
                                                    [deltaVy],
                                                    [deltaTheta]])
    F_p = np.array([[1, 0, deltaT * np.cos(alpha), 0, ((-1/2 * (deltaT ** 2) * aceleracao_medida - vx_kmenos1 * deltaT) * np.sin(alpha))],
                     [0, 1, 0, deltaT * np.sin(alpha), ((1/2 * (deltaT ** 2) * aceleracao_medida + vy_kmenos1 * deltaT) * np.cos(alpha))],
                     [0, 0, 1, 0, (-1)*(aceleracao_medida * deltaT * np.sin(alpha))],
                     [0, 0, 0, 1, (aceleracao_medida * deltaT * np.cos(alpha))],
                     [0, 0, 0, 0, 1]])
    
    F_u = np.array([[1/2 * (deltaT ** 2) * np.cos(alpha), 
                        ((-1/2 * (deltaT ** 3) * aceleracao_medida) - (vx_kmenos1 * (deltaT ** 2))) * np.sin(alpha)],
                     [1/2 * (deltaT ** 2) * np.sin(alpha),
                         ((1/2 * (deltaT ** 3) * aceleracao_medida) + (vy_kmenos1 * (deltaT ** 2))) * np.cos(alpha)],
                     [deltaT * np.cos(alpha), (-1)*(aceleracao_medida * deltaT * np.sin(alpha))],
                     [deltaT * np.sin(alpha), (aceleracao_medida * deltaT * np.cos(alpha))],
                     [0, deltaT]])
    
    Q = np.array([[1, 0],
                   [0, 1]])
    P_predito = (F_p.dot(P)).dot(F_p.transpose()) + (F_u.dot(Q)).dot(F_u.transpose())
    Sx, Sy = 1, 1
    hDF = determinarDF(posicao_predita[0][0], posicao_predita[1][0], posicao_predita[4][0], Sx, Sy)
    hDR = determinarDR(posicao_predita[0][0], posicao_predita[1][0], posicao_predita[4][0], Sx, Sy)
    hDL = determinarDL(posicao_predita[0][0], posicao_predita[1][0], posicao_predita[4][0], Sx, Sy)
    funcaoH = [hDF, hDR, hDL, 0, 0]
    distancias_lidas.append(0)
    distancias_lidas.append(0)
    Vk = np.array([distancias_lidas]).transpose() - np.array([funcaoH]).transpose()
    H = determinarMatrizH(posicao_predita[0][0], posicao_predita[1][0], posicao_predita[4][0], Sx, Sy)
    Rk = np.array([[0.01, 0, 0, 0, 0],
                    [0, 0.01, 0, 0, 0],
                    [0, 0, 0.01, 0, 0],
                    [0, 0, 0, 0.01, 0],
                    [0, 0, 0, 0, 0.01]])
    Sk = (H.dot(P_predito)).dot(H.transpose()) + Rk
    inversoSk = calculateInverseSk(Sk)
    Kk = (P_predito.dot(H.transpose())).dot(inversoSk) 
    posicao_atual = posicao_predita + (Kk.dot(Vk))
    P_atual = P_predito + ((Kk.dot(Sk)).dot(Kk.transpose()))
    return posicao_atual, P_atual

In [245]:
P = np.matrix([[0.1, 0, 0, 0, 0], [0, 0.1, 0, 0, 0], [0, 0, 0.1, 0, 0], [0, 0, 0, 0.1, 0], [0, 0, 0, 0, 0.1]])
posicao_atual, P_atual = filtroDeKalman([0, 0, 0, 0, 0], 1, [0.1, 0], P, [1, 1, 0])
print(posicao_atual)
print("-----------------------------------------------------------------------")
print(P_atual)

[[array([[0.05]])]
 [array([[0.]])]
 [array([[0.1]])]
 [array([[0.]])]
 [array([[0.]])]]
-----------------------------------------------------------------------
[[array([[0.89505495]]) array([[0.]]) array([[1.19340659]]) array([[0.]])
  array([[0.]])]
 [array([[0.]]) array([[0.19638692]]) array([[0.]]) array([[0.0105122]])
  array([[0.10512195]])]
 [array([[1.19340659]]) array([[0.]]) array([[1.89120879]]) array([[0.]])
  array([[0.]])]
 [array([[0.]]) array([[0.0105122]]) array([[0.]]) array([[0.11126829]])
  array([[0.11268293]])]
 [array([[0.]]) array([[0.10512195]]) array([[0.]]) array([[0.11268293]])
  array([[1.12682927]])]]
